In [ ]:
import math
import seaborn as sns
from sklearn import tree
import numpy as np 
import pandas as pd 
import seaborn as sns   
from sklearn import tree

In [ ]:
import plotly.express as px 
import matplotlib.pyplot as plt    
from scipy.stats import loguniform
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler 
from sklearn.model_selection import train_test_split, GridSearchCV  
from sklearn.metrics import  accuracy_score, confusion_matrix, classification_report, roc_auc_score 
from sklearn.linear_model import LogisticRegression    
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier     
from sklearn.ensemble import RandomForestClassifier     

In [ ]:
sns.set()

In [ ]:
df = pd.read_csv("C:/Users/sri03/Downloads/archive (9).zip")
df.head(5)

In [ ]:
print("Shape of the data:- ",df.shape)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
df.describe(include="all")

In [ ]:
df.isnull().sum()

In [ ]:
def count_plot(feature):
    sns.countplot(x=feature, data=df)
    plt.show()
    print("\n")

In [ ]:
columns = ['Survived','Pclass','Sex','SibSp','Embarked', 'Parch']
for i in columns:
    count_plot(i)

In [ ]:
df["Age"].plot(kind='hist', title = "Age")

In [ ]:
df.groupby('Pclass')['Survived'].value_counts()

In [ ]:
df.groupby('Embarked')['Survived'].value_counts()

In [ ]:
survived_counts = df['Survived'].value_counts().reset_index()
survived_counts.columns = ['Survived', 'Count']
fig = px.pie(survived_counts, values='Count', names=['No', 'Yes'], title='Survived', labels={'Count': 'Count'}, color = ['No', 'Yes'])
fig.update_traces(textposition='inside',  textinfo='percent+label+value')
fig.update_layout(uniformtext_minsize=14, uniformtext_mode='hide')
fig.show()

In [ ]:
fig1 = px.histogram(df, x='Sex', color='Survived', barmode='group', color_discrete_map={0: "red", 1: "blue"})
fig1.update_layout(title='Sex: Survived vs Dead')
fig1.show()

In [ ]:
fig2 = px.histogram(df, x='Pclass', color='Survived', barmode='group', title='Pclass: Survived vs Dead', labels={'Pclass': 'Pclass'}, color_discrete_map={0: 'red', 1: 'blue'})
fig2.update_layout(title='PClass: Survived vs Dead')
fig2.show()

In [ ]:
df.head()

In [ ]:
df_clean = df.drop(['PassengerId', 'Name', 'Cabin', 'Ticket'], axis=1)
df_clean.shape

In [ ]:
df_clean.head()

In [ ]:
df_clean.drop_duplicates(inplace=True)
df_clean.reset_index().shape

In [ ]:
df_clean['Age'].fillna(df['Age'].mean(), inplace=True)
df_clean.isna().sum()

In [ ]:
sex_map = {'male': 1, 'female': 2}
df_clean['Sex'] = df_clean['Sex'].map( sex_map).astype(int)

In [ ]:
df_clean['Sex'].unique()

In [ ]:
df_clean["Embarked"].fillna(df_clean["Embarked"].mode()[0], inplace=True)
df_clean.info()

In [ ]:
Embarked_map = {'S': 1, 'C': 2, 'Q': 3}
df_clean['Embarked'] = df_clean['Embarked'].map(Embarked_map).astype(int)
df_clean['Embarked'].unique()

In [ ]:
for i in [i for i in df_clean.columns]:
    if df_clean[i].nunique()>=12:
        Q1 = df_clean[i].quantile(0.20)
        Q3 = df_clean[i].quantile(0.80)
        IQR = Q3 - Q1
        IQR = Q3 - Q1
        df_clean = df_clean[df_clean[i] <= (Q3+(1.5*IQR))]
        df_clean = df_clean[df_clean[i] >= (Q1-(1.5*IQR))]
df_clean = df_clean.reset_index(drop=True)
df_clean.info()

In [ ]:
sns.pairplot(df_clean, hue='Survived')

In [ ]:
plt.figure(figsize=(20,18)) 
sns.heatmap(df_clean.corr(), annot=True, linewidths=.5) 

In [ ]:
target_corr = df_clean.corr()['Survived'].abs().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
sns.barplot(x=target_corr.index[1:], y=target_corr.values[1:])
plt.xticks(rotation=45, ha='right')
plt.xlabel('Features')
plt.ylabel('Correlation with diagnosis')
plt.title('Correlation between diagnosis and Features')
plt.tight_layout()
plt.show()

In [ ]:
X = df_clean.drop("Survived", axis=1)
y = df_clean["Survived"]

print(f"'X' shape: {X.shape}")
print(f"'y' shape: {y.shape}")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle = True, random_state=1)

In [ ]:
std = StandardScaler()
X_train = std.fit_transform(X_train)
X_test = std.transform(X_test )

In [ ]:
pd.DataFrame(X_train,columns = X.columns ).describe(include = 'all')

In [ ]:
def model_stats(y_pred, y_test):
    
    result = np.vstack((y_pred, y_test)).T
    #print(result)
    differences = np.count_nonzero(result.sum(axis = 1) == 1 )
    print('Wrong Predictions = ',differences)
    cm = confusion_matrix(y_test, y_pred)
    print(cm, '\n Accuracy Score = ',accuracy_score(y_test, y_pred))
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

In [ ]:
classifier = LogisticRegression(random_state = 0)
classifier.fit(X_train, y_train)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
model_stats(y_test, y_pred)

In [ ]:
param_grid = {
    'solver': ['lbfgs', 'newton-cg', 'sag', 'saga'],
    'penalty': ['l1', 'l2'],
    'C': [1, 10, 100]
}

classifier = LogisticRegression()
grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_train, y_train)
if grid_search.best_params_ is not None:
    best_params = grid_search.best_params_
    classifier = LogisticRegression(**best_params)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    
    model_stats(y_pred,y_test)
else:
    print('The best parameters have not been set yet.')

In [ ]:
param_grid = {
    "n_neighbors": [i for i in range(1,20, 1)],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'cosine']
}

knn = KNeighborsClassifier()
grid_search = GridSearchCV(knn, param_grid,  refit=True, verbose=1, cv=5)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
knn = KNeighborsClassifier(**best_params)
knn.fit(X_train,y_train)
y_pred = knn.predict(X_test)
model_stats(y_test,y_pred)

In [ ]:
param_grid = {"n_neighbors": [i for i in range(1,20, 1)],
             "weights": ["uniform", "distance"],
             "algorithm": ["ball_tree", "kd_tree", "brute"],
             "leaf_size": [1, 5, 10 ],
             "p": [1,2]}
knn = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
grid_search = GridSearchCV(knn, param_grid,  refit=True, verbose=1, cv=5)
grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
knn = KNeighborsClassifier(**best_params)
knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)


model_stats(y_test,y_pred)

In [ ]:
param_grid = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': [3, 5, 7, None],
    "max_features": [i for i in range(1, 10, 1)],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [i for i in range(1, 5, 1)],
    'max_leaf_nodes': [None, 10, 20],
    'min_impurity_decrease': [1e-7, 1e-5, 1e-3]
}

classifier = DecisionTreeClassifier(random_state = 0)

grid_search = GridSearchCV(classifier, param_grid, cv=5)

grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_

classifier = DecisionTreeClassifier(**best_params)
classifier.fit(X_train,y_train)

y_pred = classifier.predict(X_test)

model_stats(y_test,y_pred)

In [ ]:
param_grid = {'bootstrap': [True, False],
    'n_estimators': [30,50,100],
    'max_depth': [10, 50, 100, None],
    'min_samples_split': [2, 5, 15, 30],
    'min_samples_leaf': [1, 3, 5, 10]
}

classifier = RandomForestClassifier()
grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
classifier = RandomForestClassifier(**best_params)
classifier.fit(X_train,y_train)


y_pred = classifier.predict(X_test)

model_stats(y_test,y_pred)

In [ ]:
from sklearn.svm import SVC

param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1],
    'kernel': ['linear', 'rbf']
}

classifier = SVC()

grid_search = GridSearchCV(classifier, param_grid, cv=5)
grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_
classifier = SVC(**best_params)
classifier.fit(X_train,y_train)
y_pred = classifier.predict(X_test)

model_stats(y_test,y_pred)
